In [11]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from generation import MarkedIntensityHomogenuosPoisson, generate_samples_marked

class PointProcessDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
        self.max_len = max(len(seq) for seq in sequences)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        padded_seq = np.zeros((self.max_len, 2))
        padded_seq[:len(seq)] = np.array(seq)
        seq_len = len(seq)
        
        return {
            'sequence': torch.FloatTensor(padded_seq),
            'length': torch.LongTensor([seq_len]),
            'time_series': torch.ones(self.max_len, NUM_STEPS_TIMESERIES, TIMESERIES_FEATURE)
        }

DIM_SIZE = 7
mi = MarkedIntensityHomogenuosPoisson(DIM_SIZE)
for u in range(DIM_SIZE):
    mi.initialize(1.0, u)
simulated_sequences = generate_samples_marked(mi, 15.0, 1000)
dataset = PointProcessDataset(simulated_sequences)


In [20]:
len(simulated_sequences[3])

135

In [123]:
class PointProcessDataset(Dataset):
    def __init__(self, sequences):
        # 直接将序列转换为tensor格式存储
        self.sequences = [
            torch.tensor([[event[0], event[1]] for event in seq], dtype=torch.float32)
            for seq in sequences
        ]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        # 直接返回原始序列，不进行padding
        return self.sequences[idx]
def collate_fn(batch):
    return batch
DIM_SIZE = 7
BATCH_SIZE=256
mi = MarkedIntensityHomogenuosPoisson(DIM_SIZE)
for u in range(DIM_SIZE):
    mi.initialize(1.0, u)
simulated_sequences = generate_samples_marked(mi, 15.0, 1000)
dataset = PointProcessDataset(simulated_sequences)


# 在创建DataLoader时使用自定义的collate_fn
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn
)


In [124]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGRU(nn.Module):
    def __init__(self, num_classes, hidden_size_event, hidden_size_time, reg=0.1):
        super(SimpleGRU, self).__init__()
        self.num_classes = num_classes
        self.reg = reg
        self.event_embedding = nn.Embedding(num_classes, num_classes)
        self.event_gru = nn.GRU(num_classes, hidden_size_event, batch_first=True)
        self.time_gru = nn.GRU(1, hidden_size_time, batch_first=True)
        combined_size = hidden_size_event + hidden_size_time
        self.time_output = nn.Linear(combined_size, 1)
        self.mark_output = nn.Linear(combined_size, num_classes)

    def forward(self, event_sequence):
        marks = event_sequence[..., 0].long()
        times = event_sequence[..., 1].unsqueeze(-1)
        
        # 修正: 使用event_embedding处理marks
        mark_embedded = self.event_embedding(marks)
        event_output, _ = self.event_gru(mark_embedded)
        time_output, _ = self.time_gru(times)
        
        combined_output = torch.cat([event_output, time_output], dim=-1)
        time_pred = self.time_output(combined_output)
        mark_logits = self.mark_output(combined_output)
        return time_pred, mark_logits

    def compute_loss(self, time_pred, mark_logits, targets):
        true_times = targets[..., 1].unsqueeze(-1)
        true_marks = targets[..., 0].long()
        time_loss = F.mse_loss(time_pred, true_times)
        mark_logits_flat = mark_logits.view(-1, self.num_classes)
        true_marks_flat = true_marks.view(-1)
        mark_loss = F.cross_entropy(mark_logits_flat, true_marks_flat)
        
        total_loss = mark_loss + self.reg * time_loss
        
        return total_loss, mark_loss, time_loss


In [126]:
def train_model(model, train_loader, num_epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    训练模型的函数
    
    Args:
        model: SimpleGRU模型实例
        train_loader: 训练数据的DataLoader
        num_epochs: 训练轮数
        learning_rate: 学习率
        device: 训练设备（'cuda'或'cpu'）
    """
    print(f"Training on {device}")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_total_loss = 0
        train_mark_loss_sum = 0
        train_time_loss_sum = 0
        num_batches = 0
        
        epoch_start_time = time.time()
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch in train_bar:
            optimizer.zero_grad()
            batch_loss = 0
            batch_mark_loss = 0
            batch_time_loss = 0
            
            for sequence in batch:
                sequence = sequence.to(device)
                sequence = sequence.unsqueeze(0)
                time_pred, mark_logits = model(sequence)
                loss, mark_loss, time_loss = model.compute_loss(time_pred, mark_logits, sequence)
                batch_loss += loss
                batch_mark_loss += mark_loss
                batch_time_loss += time_loss
            batch_size = len(batch)
            batch_loss /= batch_size
            batch_mark_loss /= batch_size
            batch_time_loss /= batch_size
            batch_loss.backward()
            optimizer.step()
            train_total_loss += batch_loss.item()
            train_mark_loss_sum += batch_mark_loss.item()
            train_time_loss_sum += batch_time_loss.item()
            num_batches += 1
            
            train_bar.set_postfix({
                'loss': f'{batch_loss.item():.4f}',
                'mark_loss': f'{batch_mark_loss.item():.4f}',
                'time_loss': f'{batch_time_loss.item():.4f}'
            })
        avg_train_loss = train_total_loss / num_batches
        avg_train_mark_loss = train_mark_loss_sum / num_batches
        avg_train_time_loss = train_time_loss_sum / num_batches
        
        epoch_time = time.time() - epoch_start_time
        print(f'\nEpoch {epoch+1}/{num_epochs} - {epoch_time:.2f}s')
        print(f'Train Loss: {avg_train_loss:.4f} (Mark: {avg_train_mark_loss:.4f}, Time: {avg_train_time_loss:.4f})')
        print('-' * 80)
if __name__ == "__main__":
    # 创建模型
    model = SimpleGRU(
        num_classes=DIM_SIZE,
        hidden_size_event=16,
        hidden_size_time=32
    )
    
    # 创建训练数据加载
    
    # 训练模型
    train_model(
        model=model,
        train_loader=dataloader,
        num_epochs=10,
        learning_rate=0.001
    )
marked_intensity

Training on cpu


Epoch 1/10:   0%|          | 0/4 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [127]:
import numpy as np

################################### for neural network modeling
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [133]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# 生成训练和测试数据
T_train = np.random.exponential(size=80000).cumsum()
T_test = np.random.exponential(size=20000).cumsum()

# 计算标准化参数
mu = np.log(np.ediff1d(T_train)).mean()
sigma = np.log(np.ediff1d(T_train)).std()

class PointProcessDataset(Dataset):
    def __init__(self, times, time_step):
        self.times = times
        self.time_step = time_step
        
    def __len__(self):
        return len(self.times) - self.time_step - 1
        
    def __getitem__(self, idx):
        # 获取历史事件序列
        history = self.times[idx:idx+self.time_step]
        history = np.diff(history)  # 计算事件间隔
        history = history.reshape(-1, 1)
        
        # 获取经过的时间
        elapsed = self.times[idx+self.time_step+1] - self.times[idx+self.time_step]
        
        return {
            'history': torch.FloatTensor(history),
            'elapsed': torch.FloatTensor([elapsed]).clone()
        }
class RNNPointProcess(nn.Module):
    def __init__(self, time_step=20, size_rnn=64, size_nn=64, size_layer_chfn=2):
        super(RNNPointProcess, self).__init__()
        self.time_step = time_step
        self.size_rnn = size_rnn
        self.size_nn = size_nn
        
        # RNN层
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=size_rnn,
            batch_first=True,
            nonlinearity='tanh'  # 明确指定非线性函数
        )
        
        # 第一个隐藏层
        self.elapsed_time_linear = nn.Linear(1, size_nn, bias=False)
        self.rnn_linear = nn.Linear(size_rnn, size_nn)
        
        # 后续隐藏层
        self.hidden_layers = nn.ModuleList([
            nn.Linear(size_nn, size_nn)
            for _ in range(size_layer_chfn-1)
        ])
        
        # 输出层
        self.output_layer = nn.Linear(size_nn, 1)
        
        # 初始化为正权重
        self.apply(self._init_weights)
        
        # 注册标准化参数
        self.register_buffer('mu', torch.tensor(mu, dtype=torch.float32))
        self.register_buffer('sigma', torch.tensor(sigma, dtype=torch.float32))
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # 使用较小的初始值
            nn.init.uniform_(module.weight, 0, 0.1)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
                
    def forward(self, event_history, elapsed_time):
        # 添加数值稳定性检查
        event_history = torch.clamp(event_history, min=1e-6)
        elapsed_time = torch.clamp(elapsed_time, min=1e-6)
        elapsed_time = elapsed_time.requires_grad_(True)
        
        # 标准化输入
        event_history_norm = (torch.log(event_history) - self.mu) / self.sigma
        elapsed_time_norm = (torch.log(elapsed_time) - self.mu) / self.sigma
        
        # 检查并处理无效值
        event_history_norm = torch.nan_to_num(event_history_norm, nan=0.0, posinf=1.0, neginf=-1.0)
        elapsed_time_norm = torch.nan_to_num(elapsed_time_norm, nan=0.0, posinf=1.0, neginf=-1.0)
        
        # RNN处理历史序列
        rnn_out, _ = self.rnn(event_history_norm)
        rnn_out = rnn_out[:, -1, :]
        
        # 第一个隐藏层
        hidden_tau = self.elapsed_time_linear(elapsed_time_norm)
        hidden_rnn = self.rnn_linear(rnn_out)
        hidden = torch.tanh(hidden_tau + hidden_rnn)
        
        # 后续隐藏层
        for layer in self.hidden_layers:
            hidden = torch.tanh(layer(hidden))
        
        # 计算累积危险函数（添加数值稳定性）
        Int_l = F.softplus(self.output_layer(hidden)) + 1e-6
        
        # 计算危险函数
        try:
            l = torch.autograd.grad(Int_l.sum(), elapsed_time, create_graph=True)[0]
            # 确保l是正的且有界
            l = torch.clamp(l, min=1e-6, max=1e6)
        except RuntimeError:
            print("Gradient computation failed")
            l = torch.ones_like(elapsed_time) * 1e-6
        
        return l, Int_l
    
    def compute_loss(self, l, Int_l):
        # 添加数值稳定性
        l = torch.clamp(l, min=1e-6)
        Int_l = torch.clamp(Int_l, min=0.0)
        
        # 计算负对数似然，添加梯度裁剪
        loss = -torch.mean(torch.log(l) - Int_l)
        
        # 检查loss是否为nan
        if torch.isnan(loss):
            print("Warning: Loss is NaN!")
            loss = torch.tensor(0.0, requires_grad=True, device=l.device)
            
        return loss

def train_model(model, train_loader, test_loader=None, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on {device}")
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # 添加学习率调度器
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in train_bar:
            history = batch['history'].to(device)
            elapsed = batch['elapsed'].to(device)
            
            optimizer.zero_grad()
            
            # 前向传播
            l, Int_l = model(history, elapsed)
            loss = model.compute_loss(l, Int_l)
            
            # 检查loss是否为nan
            if not torch.isnan(loss):
                loss.backward()
                # 添加梯度裁剪
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            
            train_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_train_loss = total_loss / num_batches
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print(f'Average Train Loss: {avg_train_loss:.4f}')
        
        # 更新学习率
        scheduler.step(avg_train_loss)

# 修改超参数
if __name__ == "__main__":
    time_step = 20
    size_rnn = 32  # 减小RNN大小
    size_nn = 32   # 减小隐藏层大小
    size_layer_chfn = 2
    batch_size = 16  # 减小批量大小
    num_epochs = 20
    learning_rate = 0.0001  # 使用更小的学习率
    
    # 其余代码保持不变...
    
    # 创建数据集
    train_dataset = PointProcessDataset(T_train, time_step)
    test_dataset = PointProcessDataset(T_test, time_step)
    
    # 创建数据加载器
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    
    # 创建模型
    model = RNNPointProcess(
        time_step=time_step,
        size_rnn=size_rnn,
        size_nn=size_nn,
        size_layer_chfn=size_layer_chfn
    )
    
    # 训练模型
    train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        num_epochs=num_epochs
    )
    
    # 保存模型
    torch.save(model.state_dict(), 'rnn_point_process.pth')

/Users/wenqing_liu/ENTER/envs/pymc_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training on cpu


Epoch 1/20 [Train]: 100%|██████████| 4999/4999 [00:15<00:00, 314.21it/s, loss=1.4620]



Epoch 1/20
Average Train Loss: 1.0540


Epoch 2/20 [Train]: 100%|██████████| 4999/4999 [00:15<00:00, 327.60it/s, loss=1.2577]


...


Epoch 6/20
Average Train Loss: 1.0059


Epoch 7/20 [Train]: 100%|██████████| 4999/4999 [00:17<00:00, 290.16it/s, loss=1.6346]



Epoch 7/20
Average Train Loss: 1.0055


Epoch 8/20 [Train]:  77%|███████▋  | 3832/4999 [00:13<00:04, 283.86it/s, loss=0.8539]


KeyboardInterrupt: 

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

# 数据集类
class PointProcessDataset(Dataset):
    def __init__(self, data, time_step=20):
        self.data = data
        self.time_step = time_step
        
    def __len__(self):
        return len(self.data) - self.time_step
        
    def __getitem__(self, idx):
        history = self.data[idx:idx+self.time_step].reshape(-1, 1)
        elapsed = self.data[idx+self.time_step].reshape(-1)
        
        return {
            'history': torch.FloatTensor(history),
            'elapsed': torch.FloatTensor(elapsed)
        }

class GRUPointProcess(nn.Module):
    def __init__(self, time_step=20, size_gru=64, size_nn=64, size_layer_chfn=2):
        super(GRUPointProcess, self).__init__()
        self.time_step = time_step
        self.size_gru = size_gru
        self.size_nn = size_nn
        
        # GRU层
        self.gru = nn.GRU(
            input_size=1,
            hidden_size=size_gru,
            num_layers=1,
            batch_first=True
        )
        
        # 第一个隐藏层
        self.elapsed_time_linear = nn.Linear(1, size_nn, bias=False)
        self.gru_linear = nn.Linear(size_gru, size_nn)
        
        # 后续隐藏层
        self.hidden_layers = nn.ModuleList([
            nn.Linear(size_nn, size_nn)
            for _ in range(size_layer_chfn-1)
        ])
        
        # 输出层
        self.output_layer = nn.Linear(size_nn, 1)
        
        # 初始化为正权重
        self.apply(self._init_weights)
        
        # 注册标准化参数
        self.register_buffer('mu', torch.tensor(mu, dtype=torch.float32))
        self.register_buffer('sigma', torch.tensor(sigma, dtype=torch.float32))
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # 使用较小的初始值
            nn.init.uniform_(module.weight, 0, 0.1)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
                
    def forward(self, event_history, elapsed_time):
        # 添加数值稳定性检查
        event_history = torch.clamp(event_history, min=1e-6)
        elapsed_time = torch.clamp(elapsed_time, min=1e-6)
        elapsed_time = elapsed_time.requires_grad_(True)
        
        # 标准化输入
        event_history_norm = (torch.log(event_history) - self.mu) / self.sigma
        elapsed_time_norm = (torch.log(elapsed_time) - self.mu) / self.sigma
        
        # 检查并处理无效值
        event_history_norm = torch.nan_to_num(event_history_norm, nan=0.0, posinf=1.0, neginf=-1.0)
        elapsed_time_norm = torch.nan_to_num(elapsed_time_norm, nan=0.0, posinf=1.0, neginf=-1.0)
        
        # GRU处理历史序列
        gru_out, _ = self.gru(event_history_norm)
        gru_out = gru_out[:, -1, :]  # 取最后一个时间步的输出
        
        # 第一个隐藏层
        hidden_tau = self.elapsed_time_linear(elapsed_time_norm)
        hidden_gru = self.gru_linear(gru_out)
        hidden = torch.tanh(hidden_tau + hidden_gru)
        
        # 后续隐藏层
        for layer in self.hidden_layers:
            hidden = torch.tanh(layer(hidden))
        
        # 计算累积危险函数（添加数值稳定性）
        Int_l = F.softplus(self.output_layer(hidden)) + 1e-6
        
        # 计算危险函数
        try:
            l = torch.autograd.grad(Int_l.sum(), elapsed_time, create_graph=True)[0]
            # 确保l是正的且有界
            l = torch.clamp(l, min=1e-6, max=1e6)
        except RuntimeError:
            print("Gradient computation failed")
            l = torch.ones_like(elapsed_time) * 1e-6
        
        return l, Int_l
    
    def compute_loss(self, l, Int_l):
        # 添加数值稳定性
        l = torch.clamp(l, min=1e-6)
        Int_l = torch.clamp(Int_l, min=0.0)
        
        # 计算负对数似然，添加梯度裁剪
        loss = -torch.mean(torch.log(l) - Int_l)
        
        # 检查loss是否为nan
        if torch.isnan(loss):
            print("Warning: Loss is NaN!")
            loss = torch.tensor(0.0, requires_grad=True, device=l.device)
            
        return loss

def train_model(model, train_loader, test_loader=None, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on {device}")
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # 添加学习率调度器
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5, 
        patience=3, 
        verbose=True
    )
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in train_bar:
            history = batch['history'].to(device)
            elapsed = batch['elapsed'].to(device)
            
            optimizer.zero_grad()
            
            # 前向传播
            l, Int_l = model(history, elapsed)
            loss = model.compute_loss(l, Int_l)
            
            # 检查loss是否为nan
            if not torch.isnan(loss):
                loss.backward()
                # 移除梯度裁剪
                optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            
            train_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_train_loss = total_loss / num_batches
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print(f'Average Train Loss: {avg_train_loss:.4f}')
        
        # 更新学习率
        scheduler.step(avg_train_loss)
        
        # 保存模型
        if (epoch + 1) % 5 == 0:
            torch.save(model.state_dict(), f'gru_point_process_epoch_{epoch+1}.pt')
# 生成训练数据
def generate_data(num_samples, lambda_=1.0):
    return np.random.exponential(1/lambda_, num_samples)

# 主程序
if __name__ == "__main__":
    # 生成数据
    num_samples = 10000
    T_train = generate_data(num_samples)
    T_test = generate_data(num_samples // 10)
    
    # 计算标准化参数
    mu = np.mean(np.log(T_train))
    sigma = np.std(np.log(T_train))
    
    # 创建数据加载器
    time_step = 20
    batch_size = 32
    train_dataset = PointProcessDataset(T_train, time_step)
    test_dataset = PointProcessDataset(T_test, time_step)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True
    )
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False
    )
    
    # 创建模型
    model = GRUPointProcess(
        time_step=time_step,
        size_gru=32,
        size_nn=32,
        size_layer_chfn=2
    )
    
    # 训练模型
    train_model(
        model,
        train_loader,
        test_loader,
        num_epochs=20,
        learning_rate=0.0001
    )

Training on cpu


Epoch 1/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 186.17it/s, loss=1.7882]



Epoch 1/20
Average Train Loss: 2.4767


Epoch 2/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 200.08it/s, loss=1.2375]



Epoch 2/20
Average Train Loss: 1.6100


...


Epoch 3/20
Average Train Loss: 1.2560


Epoch 4/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 195.06it/s, loss=1.0262]



Epoch 4/20
Average Train Loss: 1.1403


Epoch 5/20 [Train]:  52%|█████▏    | 162/312 [00:00<00:00, 196.18it/s, loss=1.0139]


KeyboardInterrupt: 

In [153]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

# 生成训练数据
def generate_data(num_samples, lambda_=0.5):
    # 生成指数分布的时间间隔
    intervals = np.random.exponential(1/lambda_, num_samples)
    # 计算累积时间
    times = np.cumsum(intervals)
    return times

# 数据集类
class PointProcessDataset(Dataset):
    def __init__(self, data, time_step=20):
        # 确保数据是按时间排序的
        self.data = np.sort(data)
        self.time_step = time_step
        
    def __len__(self):
        return len(self.data) - self.time_step
        
    def __getitem__(self, idx):
        # 获取时间序列窗口
        history = self.data[idx:idx+self.time_step].reshape(-1, 1)
        next_time = self.data[idx+self.time_step]
        
        # 计算时间差（elapsed time）
        elapsed = next_time - history[-1, 0]
        
        return {
            'history': torch.FloatTensor(history),
            'elapsed': torch.FloatTensor([elapsed])
        }

class GRUPointProcess(nn.Module):
    def __init__(self, time_step=20, size_gru=64, size_nn=64, size_layer_chfn=2):
        super(GRUPointProcess, self).__init__()
        self.time_step = time_step
        self.size_gru = size_gru
        self.size_nn = size_nn
        
        # GRU层
        self.gru = nn.GRU(
            input_size=1,
            hidden_size=size_gru,
            num_layers=1,
            batch_first=True
        )
        
        # 第一个隐藏层
        self.elapsed_time_linear = nn.Linear(1, size_nn, bias=False)
        self.gru_linear = nn.Linear(size_gru, size_nn)
        
        # 后续隐藏层
        self.hidden_layers = nn.ModuleList([
            nn.Linear(size_nn, size_nn)
            for _ in range(size_layer_chfn-1)
        ])
        
        # 输出层
        self.output_layer = nn.Linear(size_nn, 1)
        
        # 初始化为正权重
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # 使用较小的初始值
            nn.init.uniform_(module.weight, 0, 0.1)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
                
    def forward(self, event_history, elapsed_time):
        # 添加数值稳定性检查
        event_history = torch.clamp(event_history, min=1e-6)
        elapsed_time = torch.clamp(elapsed_time, min=1e-6)
        elapsed_time = elapsed_time.requires_grad_(True)
        
        # 标准化输入
        event_history_norm = event_history
        elapsed_time_norm = elapsed_time
        
        # GRU处理历史序列
        gru_out, _ = self.gru(event_history_norm)
        gru_out = gru_out[:, -1, :]  # 取最后一个时间步的输出
        
        # 第一个隐藏层
        hidden_tau = self.elapsed_time_linear(elapsed_time_norm)
        hidden_gru = self.gru_linear(gru_out)
        hidden = torch.tanh(hidden_tau + hidden_gru)
        
        # 后续隐藏层
        for layer in self.hidden_layers:
            hidden = torch.tanh(layer(hidden))
        
        # 计算累积危险函数（添加数值稳定性）
        Int_l = F.softplus(self.output_layer(hidden)) + 1e-6
        
        # 计算危险函数
        try:
            l = torch.autograd.grad(Int_l.sum(), elapsed_time, create_graph=True)[0]
            # 确保l是正的且有界
            l = torch.clamp(l, min=1e-6, max=1e6)
        except RuntimeError:
            print("Gradient computation failed")
            l = torch.ones_like(elapsed_time) * 1e-6
        
        return l, Int_l
    
    def compute_loss(self, l, Int_l):
        # 添加数值稳定性
        l = torch.clamp(l, min=1e-6)
        Int_l = torch.clamp(Int_l, min=0.0)
        
        # 计算负对数似然
        loss = -torch.mean(torch.log(l) - Int_l)
        
        # 检查loss是否为nan
        if torch.isnan(loss):
            print("Warning: Loss is NaN!")
            loss = torch.tensor(0.0, requires_grad=True, device=l.device)
            
        return loss

def train_model(model, train_loader, test_loader=None, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on {device}")
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # 添加学习率调度器
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5, 
        patience=3, 
        verbose=True
    )
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in train_bar:
            history = batch['history'].to(device)
            elapsed = batch['elapsed'].to(device)
            
            optimizer.zero_grad()
            
            # 前向传播
            l, Int_l = model(history, elapsed)
            loss = model.compute_loss(l, Int_l)
            
            # 检查loss是否为nan
            if not torch.isnan(loss):
                loss.backward()
                optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            
            train_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_train_loss = total_loss / num_batches
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print(f'Average Train Loss: {avg_train_loss:.4f}')
        
        # 更新学习率
        scheduler.step(avg_train_loss)
        
        # 保存模型
        if (epoch + 1) % 5 == 0:
            torch.save(model.state_dict(), f'gru_point_process_epoch_{epoch+1}.pt')

# 主程序
if __name__ == "__main__":

    torch.manual_seed(42)
    np.random.seed(42)
    
    num_samples = 10000
    T_train = generate_data(num_samples)
    T_test = generate_data(num_samples // 10)
    
    # 创建数据加载器
    time_step = 20
    batch_size = 32
    train_dataset = PointProcessDataset(T_train, time_step)
    test_dataset = PointProcessDataset(T_test, time_step)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=False  # 不打乱顺序，保持时间序列的连续性
    )
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False
    )
    
    # 创建模型
    model = GRUPointProcess(
        time_step=time_step,
        size_gru=32,
        size_nn=32,
        size_layer_chfn=2
    )
    
    train_model(
        model,
        train_loader,
        test_loader,
        num_epochs=20,
        learning_rate=0.001
    )

# 检查数据形状
sample_batch = next(iter(train_loader))
print("\nData shapes:")
print("History shape:", sample_batch['history'].shape)
print("Elapsed shape:", sample_batch['elapsed'].shape)

/Users/wenqing_liu/ENTER/envs/pymc_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training on cpu


Epoch 1/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 182.09it/s, loss=2.1096]



Epoch 1/20
Average Train Loss: 2.0911


Epoch 2/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 187.04it/s, loss=1.8564]


...


Epoch 9/20
Average Train Loss: 1.6951


Epoch 10/20 [Train]: 100%|██████████| 312/312 [00:01<00:00, 195.05it/s, loss=1.8211]



Epoch 10/20
Average Train Loss: 1.6929


Epoch 11/20 [Train]:  57%|█████▋    | 178/312 [00:00<00:00, 199.47it/s, loss=1.7709]


KeyboardInterrupt: 

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

def generate_synthetic_data(n_events=100, lambda_rate=0.5):
    inter_event_times = np.random.exponential(scale=1/lambda_rate, size=n_events)
    event_times = np.cumsum(inter_event_times)
    return event_times
event_times = generate_synthetic_data(n_events=200)

# 定义数据集
class PointProcessDataset(Dataset):
    def __init__(self, data, time_step=20):
        self.data = data
        self.time_step = time_step
        
    def __len__(self):
        return len(self.data) - self.time_step
        
    def __getitem__(self, idx):
        history = self.data[idx:idx+self.time_step].reshape(-1, 1)
        next_time = self.data[idx+self.time_step]
        elapsed = next_time - history[-1, 0]
        return {
            'history': torch.FloatTensor(history),
            'elapsed': torch.FloatTensor([elapsed])
        }

# 数据准备
time_step = 20
dataset = PointProcessDataset(event_times, time_step)


In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the GRUPointProcess model
class GRUPointProcess(nn.Module):
    def __init__(self, time_step=4, size_gru=64, size_nn=64, size_layer_chfn=4):
        super(GRUPointProcess, self).__init__()
        self.time_step = time_step
        self.size_gru = size_gru
        self.size_nn = size_nn
        self.gru = nn.GRU(
            input_size=1,
            hidden_size=size_gru,
            num_layers=1,
            batch_first=True
        )
        self.elapsed_time_linear = nn.Linear(1, size_nn, bias=False)
        self.gru_linear = nn.Linear(size_gru, size_nn)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(size_nn, size_nn)
            for _ in range(size_layer_chfn-1)
        ])
        self.output_layer = nn.Linear(size_nn, 1)
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.uniform_(module.weight, 0, 0.1)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
                
    def forward(self, event_history, elapsed_time):
        event_history = torch.clamp(event_history, min=1e-6)
        elapsed_time = torch.clamp(elapsed_time, min=1e-6)
        elapsed_time = elapsed_time.requires_grad_(True)
        gru_out, _ = self.gru(event_history)
        gru_out = gru_out[:, -1, :]
        hidden_tau = self.elapsed_time_linear(elapsed_time)
        hidden_gru = self.gru_linear(gru_out)
        hidden = torch.tanh(hidden_tau + hidden_gru)
        for layer in self.hidden_layers:
            hidden = torch.tanh(layer(hidden))
        Int_l = F.softplus(self.output_layer(hidden))
        try:
            l = torch.autograd.grad(Int_l, elapsed_time, create_graph=True)[0]
            l = torch.clamp(l, min=1e-6, max=1e6)
        except RuntimeError:
            print("Gradient computation failed")
            l = torch.ones_like(elapsed_time) * 1e-6
        return l, Int_l
    
    def compute_loss(self, l, Int_l):
        l = torch.clamp(l, min=1e-6)
        Int_l = torch.clamp(Int_l, min=0.0)
        loss = -torch.mean(torch.log(l) - Int_l)
        if torch.isnan(loss):
            print("Warning: Loss is NaN!")
            loss = torch.tensor(0.0, requires_grad=True, device=l.device)
            
        return loss

# Create some dummy data
event_history = torch.randn(2, 4, 1)  # 2 samples, 4 time steps, 1 feature
elapsed_time = torch.randn(2, 1)  # 2 samples, 1 elapsed time value

# Instantiate and run the model
model = GRUPointProcess(time_step=4, size_gru=64, size_nn=64, size_layer_chfn=2)
l, Int_l = model(event_history, elapsed_time)

# Print the output shapes
print(f"Shape of predicted intensity (l): {l.shape}")
print(f"Shape of cumulative intensity (Int_l): {Int_l.shape}")


Gradient computation failed
Shape of predicted intensity (l): torch.Size([2, 1])
Shape of cumulative intensity (Int_l): torch.Size([2, 1])
